In [25]:
#!/usr/bin/python3
import tensorflow as tf
import numpy as np
from math import exp
from numpy.random import binomial
from random import shuffle
from random import seed

nx = 5
ny = 5
nz = 5
N = nx*ny*nz
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


In [5]:
# Storing the IDs of the neurons
LSM_ID = np.zeros((nx,ny,nz),dtype=np.int64)
l = 0
for i in range(nx):
    for j in range(ny):
        for k in range(nz):
            LSM_ID[i,j,k] = l
            l = l + 1

LSM_ID = tf.convert_to_tensor(LSM_ID,dtype=tf.int64)
# print(LSM_ID)

In [6]:
def ID_to_ind(nx,ny,nz,ID):
    x = int(ID/(ny*nz))
    y = int( (ID-(ny*nz)*x) / nz)
    z = int(ID%nz)
    return [x, y, z]

In [30]:
# Storing the synapse connections, and creating the initial weight matrix
seed(1)
k_prob = [0.45, 0.3, 0.6, 0.15]
r_sq = 2**2

W_arr = [3, 6, -2, -2]
W_init = 3
Weights_temp = np.zeros((N,N))

N_in = int(N*0.8)
neuron_type = [ int(i<N_in) for i in range(N)]
shuffle(neuron_type) # 1 for excitatory, 0 for inhibitory

synapes = [dict() for i in range(N)]    # an array of dictonaries which store the location of neuron, type of neuron, and the IDs of the neurons it is connected to

for l in range(N):
    loc = ID_to_ind(nx,ny,nz,l)
    n_type = neuron_type[l]
    cons = []
    for i in range(nx):
        for j in range(ny):
            for k in range(nz):
                if l != int(LSM_ID[i,j,k]):
                    dist_sq = (loc[0]-i)**2 + (loc[1]-j)**2 + (loc[2]-k)**2
                    k_probl = 0
                    if n_type == 1:
                      if neuron_type[int(LSM_ID[i,j,k])] == 1:
                        k_probl = k_prob[0]
                        W_init = W_arr[0]
                      else:
                        k_probl = k_prob[1]
                        W_init = W_arr[1]
                    else:
                      if neuron_type[int(LSM_ID[i,j,k])] == 1:
                        k_probl = k_prob[2]
                        W_init = W_arr[2]
                      else:
                        k_probl = k_prob[3]
                        W_init = W_arr[3]

                    probability = k_probl* exp(-1*dist_sq/r_sq)
                    check = binomial(1,probability)
                    if check == 1:
                        cons.append(int(LSM_ID[i,j,k]))
                        Weights_temp[l,int(LSM_ID[i,j,k])] = W_init    
    synapes[l] = {"Location":loc, "Neuron_type":n_type, "connections":cons}

global Weights
Weights = tf.convert_to_tensor(Weights_temp,dtype=tf.float64)
Delay = 1 #constant delay for all synapses

In [37]:
# from exceptions import ValueError
global vrest, vth, t_refrac
vrest, vth, t_refrac = 0, 20, 2

tau_m = 32
params_potential = {'C':1, 'g_L':1/tau_m, 'E_L':vrest, 'V_T':vth, 'R_p':t_refrac}

tau_c = 64
C_theta = 5
del_C = 3
n_bits = 3
params_conc = {'C_theta':C_theta, 'del_C':del_C, 'tau_c':64, 'nbits':n_bits}
C_theta, del_c, tau_c, nbits = params_conc.values()

#### LIF Neuron single step solver
Converted from MATLAB.  Modified version of LIF solver for HW1, with N neurons and refractory period added, solves just a single timestep 

In [ ]:
def LIF(V_neuron_prev,I_input_prev,I_input_next,N,h,index_next,index_prev_spike, params):
    C, g_L, E_L, V_T, R_p = params.values()
    R_p_ind = tf.math.ceil(R_p/h)
    
    V_neuron_next = tf.math.scalar_mul(E_L,tf.ones((N,), dtype='float'))
    Spike_next = tf.zeros((N,), dtype='int64')
    
    k1 = (1/C)*(-g_L*(V_neuron_prev-E_L)+I_input_prev)
    V_temp = V_neuron_prev + k1*h/2
    I_temp = I_input_prev/2 + I_input_next/2
    k2 = (1/C)*(-g_L*(V_temp-E_L)+I_temp)
    V_temp = V_neuron_prev + k2*h
    
    for i in range(N):
        if index_next-int(index_prev_spike[i]) < R_p_ind:
            V_neuron_next = tf.tensor_scatter_nd_update(V_neuron_next,[[i]],[[E_L]])
        elif np.float64(V_temp[i]) < V_T:
            V_neuron_next = tf.tensor_scatter_nd_update(V_neuron_next,[[i]],[[np.float64(V_temp[i])]])
        else:
            Spike_next    = tf.tensor_scatter_nd_update(V_neuron_next,[[i]],[[int(1)]]) 
            V_neuron_next = tf.tensor_scatter_nd_update(V_neuron_next,[[i]],[[np.float64(V_temp[i])]])
    
    return V_neuron_next, Spike_next
    
            

### Synaptic Current Solver
solves the current input to neuron j due to spike in neuron i

In [ ]:
def syn_res(syn_string,type_syn,t,time,i,j,w_ij,del_i,h,M):  
    # spike in neuron i, produces a synaptic current in neuron j, weight = w_ij
    shape = tf.constant([M])

    ts_ds = np.float64(time[t]) + del_i
    ind = int(tf.where(time == ts_ds))
    
    if syn_string == "static":
        indices = tf.constant([[ind]])
        updates = tf.constant([w_ij/h])
        syn_curr = tf.scatter_nd(indices, updates, shape)
       
    elif syn_string == "first-order":
        tau_s = 4 * h 
        temp = w_ij * (1/tau_s) * tf.exp(-(1/tau_s)*(time -ts_ds))
        updates = temp[ind:M]
        indices = tf.constant([[k for k in range(ind,M)]])

        syn_curr = tf.scatter_nd(indices, updates, shape)


    elif syn_string == "second-order":
        if type_syn == 1:
            tau_s1, tau_s2 = 4, 8
        elif type_syn == 0:
            tau_s1, tau_s2 = 4, 2
        temp = (w_ij/(tau_s1-tau_s2)) * (tf.exp(-(1/tau_s1)*(time -ts_ds)) -tf.exp(-(1/tau_s2)*(time -ts_ds)))
        updates = temp[ind:M]
        indices = tf.constant([[k for k in range(ind,M)]])
        syn_curr = tf.scatter_nd(indices, updates, shape)
        
            
    return syn_curr

Calcium concentration update and Weight learning by calculation of Digitized Calcium concentration updation, **Equations 15 , 16 , 17, 18 in IEEE paper**<br>
Parameters are taken from table, not sure about value of nbit yet. P+ and P- values are taken from vivek's paper(2019, IJCNN).

In [ ]:
def conc_update(prev_conc, Spike, tau_c, h):
    return prev_conc*(1 - h/tau_c) + Spike


In [ ]:
def Weight_learner(last_conc, weight_prev,
                   C_theta=5,  del_c=3, nbit=3, type_syn = None):
    """
        Set type_syn as 1 for E --> E/I and 0 for I --> E/I, basically fanout from I or E.
    """
    
    p_plus = 0.1; p_minus = 0.1;
    
    
    # if type_syn not in (1, 0): raise ValueError("Invalid type")
    
    Wmax = 8 if type_syn==0 else 8*(1 - 2**(nbit - 1))
    Wmin = -8 if type_syn==1 else -8*(1 - 2**(nbit - 1))
    del_W = 0.0002 * 2**(nbit - 4)
    
    
    
    if (C_theta < last_conc < C_theta + del_c) and (weight_prev < Wmax):
        Wnew = weight_prev + del_w if binomial(1, p_plus) == 1 else weight_prev
    elif (C_theta - del_c < last_conc < C_theta ) and (weight_prev > Wmin):
        Wnew = weight_prev - del_w if binomial(1, p_minus) == 1 else weight_prev
    else:
        Wnew = weight_prev
        
        
    return new_conc, Wnew
    

#### Reservoir solver
Converted from **MATLAB assignment 3 Q2** neuron solver <br>


In [35]:
def reservoir_solver(N, Delay, synapes, M, h, I_app,threshold, params_potential, params_conc, **kwargs):
    C_theta, del_c, tau_c, nbits = params_conc.values()
    
    global Weights

    I_syn = tf.zeros((N,M))
    I_total = tf.zeros((N,M))
    V_neurons = vrest*tf.ones((N,M)) # potential of each neuron at every time instant
    Spikes = tf.zeros((N,M))         # 1 if ith neuron spikes at jth time step
    Calcium_conc = tf.zeros((N,M))

    syn_string = "static"
    
    index_prev_spike = -1*(M)*tf.ones((N,))

    time = tf.convert_to_tensor([j*h for j in range(M)])

    for t in range(1,M):
        I_total = I_app + I_syn

        V_neuron, Spike = LIF(V_neurons[:,t-1],I_total[:,t-1],I_total[:,t],N,h,t,index_prev_spike, params_potential)  # solve for neuron potential and check if spike is produced
        indices = [[k,t] for k in range(N)]

        V_neurons = tf.tensor_scatter_nd_add(V_neurons,indices=indices,updates=V_neuron)
        Spikes = tf.tensor_scatter_nd_add(Spikes,indices=indices,updates=Spike)
        
        conc = conc_update(Calcium_conc[:,t-1], Spike, tau_c, h)
        Calcium_conc = tf.tensor_scatter_nd_add(Calcium_conc,indices=indices,updates=conc)

        for i in range(N):
            if int(Spike[i]) == 1:
                index_prev_spike = tf.tensor_scatter_nd_update(index_prev_spike,[[i]],[[1]])
                
                I_syn_additional = tf.zeros((N,M))
                neurons = synapes[i]["connections"]
                neuron_tp = synapes[i]["Neuron_type"]

                for j in range(len(neurons)): # iteration over the synapic connection from i to neurons[j]
                    updates = syn_res(syn_string,neuron_tp,t,time,i,neurons[j],np.float64(Weights[i,j]),Delay,h,M)
                    indices = [[neurons[j], k] for k in range(M) ]
                    I_syn_additional = tf.tensor_scatter_nd_add(I_syn_additional,indices=indices,updates=updates)

                    W_new = Weight_learner(last_conc, weight_prev, C_theta,  del_c, nbit, neuron_tp)
                    index = [[i,neurons[j]]]
                    Weights = tf.tensor_scatter_nd_add(Weights,indices=index, updates=[W_new])
      
        I_syn = I_syn + I_syn_additional

    
    return V_neurons, Spikes
    